In [1]:
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer
from collections import defaultdict
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
# from livelossplot import PlotLosses

In [2]:
import datasets
# train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
idx_intent = np.load('./total_idx_intent.npy', allow_pickle=True).item()
idx_titles = np.load('./total_idx_titles.npy', allow_pickle=True).item()

intents = []
titles = []
for k,v in idx_intent.items():
    intents.append(v)
    titles.append(idx_titles[k])
    
# split train, test set = 8:2
test_num = int(len(intents)*0.2)

train_intent = intents[:-test_num]
train_titles = titles[:-test_num]
test_intent = intents[-test_num:]
test_titles = titles[-test_num:]

In [3]:
train_intent.pop(1031)
train_titles.pop(1031)
test_intent.pop(816)
test_titles.pop(816)

'Nestle Hot Cocoa Mix Rich Chocolate - 70/0.75oz. Envelopes, Community Coffee Whole Bean Coffee, French Roast, 12-Ounce Bags (Pack of 3)'

In [4]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", do_lower_case=True)

In [5]:
title_intent = defaultdict(list)
# encoder_max_length=512
# decoder_max_length=32

for i in range(len(train_intent)):
    # token_titles = tokenizer(train_titles[i], add_special_tokens=False, return_tensors="pt", padding="max_length",truncation=True, max_length=encoder_max_length)
    # title_intent['titles'].append(torch.as_tensor(token_titles.input_ids, dtype=torch.int))
    # title_intent['attention_mask'].append(torch.as_tensor(token_titles.attention_mask, dtype=torch.int))
    # token_labels = tokenizer(train_intent[i], add_special_tokens=False, return_tensors="pt", padding="max_length",truncation=True, max_length=decoder_max_length)
    # title_intent['labels'].append(torch.as_tensor(token_labels.input_ids, dtype=torch.int))
    # title_intent['decoder_attention_mask'].append(torch.as_tensor(token_labels.attention_mask, dtype=torch.int))
    title_intent['titles'].append(train_titles[i])
    title_intent['intents'].append(train_intent[i])
    
test_title_intent = defaultdict(list)
for i in range(len(test_intent)):
    # token_titles = tokenizer(test_titles[i], add_special_tokens=False, return_tensors="pt", padding="max_length",truncation=True, max_length=encoder_max_length)
    # test_title_intent['titles'].append(torch.as_tensor(token_titles.input_ids, dtype=torch.int))
    # test_title_intent['attention_mask'].append(torch.as_tensor(token_titles.attention_mask, dtype=torch.int))
    # token_labels = tokenizer(test_intent[i], add_special_tokens=False, return_tensors="pt", padding="max_length",truncation=True, max_length=decoder_max_length)
    # test_title_intent['labels'].append(torch.as_tensor(token_labels.input_ids, dtype=torch.int))
    # test_title_intent['decoder_attention_mask'].append(torch.as_tensor(token_labels.attention_mask, dtype=torch.int))
    test_title_intent['titles'].append(test_titles[i])
    test_title_intent['intents'].append(test_intent[i])
    
dataset = Dataset.from_dict(title_intent)
vali_dataset = Dataset.from_dict(test_title_intent)

# dataset.set_format("torch")
# vali_dataset.set_format("torch")

In [6]:
# import pandas as pd
# from IPython.display import display, HTML
# from datasets import ClassLabel

# df = pd.DataFrame(train_data[:1])
# del df["id"]
# for column, typ in train_data.features.items():
#       if isinstance(typ, ClassLabel):
#           df[column] = df[column].transform(lambda i: typ.names[i])
# display(HTML(df.to_html()))

In [10]:
# from transformers import BertTokenizerFast
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [11]:
# def map_to_length(x):
#   x["article_len"] = len(tokenizer(x["article"]).input_ids)
#   x["article_longer_512"] = int(x["article_len"] > 512)
#   x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
#   x["summary_longer_64"] = int(x["summary_len"] > 64)
#   x["summary_longer_128"] = int(x["summary_len"] > 128)
#   return x

In [6]:
def map_to_length(x):
    x["article_len"] = len(tokenizer(x["titles"]).input_ids)
    x["article_longer_256"] = int(x["article_len"] > 256)
    x["article_longer_512"] = int(x["article_len"] > 512)
    x["summary_len"] = len(tokenizer(x["intents"]).input_ids)
    x["summary_longer_16"] = int(x["summary_len"] > 16)
    x["summary_longer_32"] = int(x["summary_len"] > 32)
    return x

In [7]:
dataset

Dataset({
    features: ['titles', 'labels'],
    num_rows: 4355
})

In [8]:
sample_size = len(train_intent)
data_stats = dataset.map(map_to_length, num_proc=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (997 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors


In [9]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Min: {}, Article Max: {}, Article Mean: {}, %-Articles > 256:{}, %-Articles > 512:{}, Summary Min:{}, Summary Max:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            min(x["article_len"]),
            max(x["article_len"]),
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_256"]) / sample_size, 
            sum(x["article_longer_512"]) / sample_size, 
            min(x["summary_len"]),
            max(x["summary_len"]),
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_16"]) / sample_size,
            sum(x["summary_longer_32"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Min: 12, Article Max: 997, Article Mean: 72.91113662456947, %-Articles > 256:0.016073478760045924, %-Articles > 512:0.0009184845005740528, Summary Min:3, Summary Max:103, Summary Mean:5.708381171067738, %-Summary > 64:0.018828932261768083, %-Summary > 128:0.005510907003444317


In [6]:
encoder_max_length=512
decoder_max_length=64

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
    inputs = tokenizer(batch["titles"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["intents"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
  # batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [7]:
# train_data = train_data.select(range(32))


In [7]:
# batch_size = 16
batch_size = 8

train_data = dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["titles", "intents"]
)

  0%|          | 0/545 [00:00<?, ?ba/s]

In [8]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)


In [13]:
# val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
# val_data = val_data.select(range(8))

In [9]:
val_data = vali_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["titles", "intents"]
)

  0%|          | 0/136 [00:00<?, ?ba/s]

In [10]:
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_attention_mask", "labels"],
)

In [11]:
from transformers import EncoderDecoderModel
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id = tokenizer.sep_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

bert2bert.config.max_length = 16
bert2bert.config.min_length = 1
bert2bert.config.no_repeat_ngram_size = 2
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4
# bert2bert.config.

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    logging_steps=500,
    save_steps=1000,
    eval_steps=500,
    learning_rate=0.00007,
    num_train_epochs=4,
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)


In [13]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    # print(pred_str)
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }


In [14]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Using amp half precision backend
***** Running training *****
  Num examples = 4355
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2180
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:524: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
500,4.660100,3.953168,0.030000,0.025600,0.023600
1000,3.575400,3.621576,0.073700,0.051000,0.056500
1500,3.031900,3.445939,0.072100,0.067700,0.067400
2000,2.573500,3.394532,0.070000,0.065100,0.065200


***** Running Evaluation *****
  Num examples = 1087
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1087
  Batch size = 8
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./checkpoint-1000/special_tokens_map.json
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:524: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, Fu

TrainOutput(global_step=2180, training_loss=3.377664352556981, metrics={'train_runtime': 1491.8884, 'train_samples_per_second': 11.676, 'train_steps_per_second': 1.461, 'total_flos': 1.06863704303616e+16, 'train_loss': 3.377664352556981, 'epoch': 4.0})

In [15]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

bert2bert = EncoderDecoderModel.from_pretrained("./checkpoint-2000/").to(device)

loading configuration file ./checkpoint-2000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "bert-base-uncased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermed

In [16]:
tokenizer = BertTokenizer.from_pretrained("./checkpoint-2000")

Didn't find file ./checkpoint-2000/added_tokens.json. We won't load it.
loading file ./checkpoint-2000/vocab.txt
loading file None
loading file ./checkpoint-2000/special_tokens_map.json
loading file ./checkpoint-2000/tokenizer_config.json
loading file ./checkpoint-2000/tokenizer.json


In [19]:
# from transformers import BertTokenizer

# bert2bert = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail").to("cuda")
# tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

In [20]:
# test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:2%]")


In [17]:
def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["titles"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = bert2bert.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_summary"] = output_str

    return batch

In [18]:
batch_size = 8  # change to 64 for full evaluation

results = vali_dataset.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["titles"])


  0%|          | 0/136 [00:00<?, ?ba/s]

### Table : Results

|      | Recall | Precision | F1 |
|:------|:------:|:----------:|:------:|
| ROUGE-1 | 22.36 | 26.36 | 22.32 |
| ROUGE-2 | 5.98 | 6.62 | 6.03 |
| ROUGE-L | 22.06 | 25.99 | 22.03 |


|      | Recall | Precision | F1 |
|:------|:------:|:----------:|:------:|
| ROUGE-1 | 13.56 | 13.93 | 12.60 |
| ROUGE-2 | 2.89 | 3.23 | 2.88 |
| ROUGE-L | 13.39 | 13.82 | 12.49 |

In [19]:
for i in range(len(results["pred_summary"])):
    print(results["pred_summary"][i])
    print('')
    print(results["intents"][i])
    print('-----')

computer accessories

assembling of computer r
-----
different adaptors

adapter
-----
computer accessories

Desktop essentials
-----
computer accessories

computer
-----
men s boots

These are all male items, shoes and clothing 
-----
different style of shoes

Alternative shoes for men 
-----
snacks

candy and snacks
-----
snacks

candies
-----
quick quality

Healthy Side
-----
all are are be are a be be the be intent

all are used as snack
-----
different style of bra

Women's bra
-----
gps accessories

Windshield 
-----
headphones

Audio accessories
-----
headphones

 Audio Cable
-----
assbling a desktop

pc building components
-----
tv cables

Using a Blu Ray player
-----
hdfi setup

Blueray player setup
-----
ipod accessories

protecting ipod
-----
apple ipad

protecting ipad
-----
different style of earrings

1,2,3,5 are relavant
-----
different style of earrings

Bijoux
-----
different style of earrings

earrings diffrent style
-----
rings

different style of rings
-----
differe

In [17]:
def compute_val_metrics(pred):
    # labels_ids = pred.intents
    # pred_ids = pred.pred_summary

    pred_str = pred.intents
    # labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = pred.pred_summary
    # print(pred_str)
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [20]:
rouge_output = rouge.compute(predictions=results["pred_summary"], references=results["intents"], rouge_types=["rouge2"])["rouge2"].mid

In [21]:
round(rouge_output.precision, 4),round(rouge_output.recall, 4),round(rouge_output.fmeasure, 4)

(0.0662, 0.0598, 0.0603)

In [22]:
rouge_output = rouge.compute(predictions=results["pred_summary"], references=results["intents"], rouge_types=["rouge1"])["rouge1"].mid

In [23]:
round(rouge_output.precision, 4),round(rouge_output.recall, 4),round(rouge_output.fmeasure, 4)

(0.2636, 0.2236, 0.2232)

In [24]:
rouge_output = rouge.compute(predictions=results["pred_summary"], references=results["intents"], rouge_types=["rougeL"])["rougeL"].mid

In [25]:
round(rouge_output.precision, 4),round(rouge_output.recall, 4),round(rouge_output.fmeasure, 4)

(0.2599, 0.2206, 0.2203)